# Inteligencja Obliczeniowa w Analizie Danych Cyfrowych

##	Projekt I

### Autorzy
- Dominik Breksa
- Robert Barcik
- Konrad Bodzioch

Download all the necessary packages to run this `.ipynb` script.

Python version: `3.12.2`
Used packages:
- *numpy*, `1.26.4`
- *pandas*, `2.8.2`
- *matplotlib*, `3.8.3`
- *easyAI*, `2.0.12`

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install easyAI

Import all the necessary packages to run this `.ipynb` script.

In [ ]:
import numpy as np
import pandas as pd

### Nimby

Nimby game created as described in the task. Probabilistic model added in make_move method.
Added typing to help with code completion and readability.

In [ ]:
from typing import Optional, Final
from random import random

from easyAI import TwoPlayerGame, AI_Player, Human_Player


mutation_probability: Final[float] = 0.1

class Nimby(TwoPlayerGame):
    def __init__(self, players: Optional[list[AI_Player | Human_Player]] = None, max_removals_per_turn: Optional[int] = None, piles: Optional[list[int]] = None) -> None:
        if piles is None:
            piles = [5, 5, 5, 5]
        self.players: Optional[AI_Player | Human_Player] = players
        self.piles: Optional[list[int]] = piles
        self.max_removals_per_turn: Optional[int] = max_removals_per_turn
        self.current_player: int = 1

    def possible_moves(self) -> list[str]:
        return [
            f'{i + 1},{j}'
            for i in range(len(self.piles))
            for j in range(
                1,
                self.piles[i] + 1
                if self.max_removals_per_turn is None
                else min(self.piles[i] + 1, self.max_removals_per_turn),
            )
        ]

    def make_move(self, move: str) -> None:
        where, count = tuple(map(int, move.split(",")))
        #   Added randomness as described in the task
        if mutation_probability >= random():
            remove: int = count - 1
        else:
            remove: int = count

        self.piles[where - 1] -= remove

    def show(self) -> None:
        print(' '.join(map(str, self.piles)))

    def win(self) -> bool:
        return max(self.piles) == 0

    def is_over(self) -> bool:
        return self.win()

    def scoring(self) -> int:
        return 100 if self.win() else 0

    def ttentry(self) -> tuple:
        """Optional, speeds up AI move computation."""
        return tuple(self.piles)

Make randomness seed deterministic.

In [ ]:
from random import seed

seed(42)

Simple test of the Nimby game, to check if it works.

In [ ]:
from easyAI import AI_Player, Negamax
from easyAI.AI import TranspositionTable

#   Test
tt: TranspositionTable = TranspositionTable()
ai1: Negamax = Negamax(8, tt=tt)
ai2: Negamax = Negamax(4, tt=tt)
game: Nimby = Nimby([AI_Player(ai1), AI_Player(ai2)])
game.play()

'player %d wins' % game.current_player

### Negamax - Deterministic

1. Firstly we solve the Nim game using the `easyAI` library, in order to see max depth (`d == 14`) for perfect AI.
2. We can also acknowledge the fact that the first player will always lose (`w == -1`), and the perfect move is `1,1`

In [ ]:
from easyAI import solve_with_iterative_deepening

from easyAI.games import Nim

tt: TranspositionTable = TranspositionTable()
w, d, m = solve_with_iterative_deepening(Nim(), range(1, 20), win_score=80, tt=tt)
w, d, m, len(tt.d)

In [ ]:
max_depth: Final[int] = d

# Create a pandas dataframe to store the results
df: pd.DataFrame = pd.DataFrame(
    columns=['game_variant', 'algorithm', 'depths', 'starting_player', 'winner', 'time', 'rounds'])

# If you have the computing power you can test all the possible combinations of depths, since the game is deterministic, you will always get the same results.
# from itertools import product
# config: list[tuple[int, int]] = list(product(range(1, max_depth + 1), repeat=2))

# Tests as described in task.
config_nim: list[tuple[int, int]] = [
    (max_depth, max_depth // 2),
    (max_depth // 3, max_depth // 2),
]

config_nim

In [ ]:
df

In [ ]:
from typing import Callable, Any
from time import perf_counter

from easyAI.AI.DUAL import DUAL
from easyAI.AI.Negamax import Negamax
from easyAI.AI.SSS import SSS

type DepthConfiguration = tuple[int, int]
type DataframeRecord = tuple[str, str, np.array, np.uint8, np.uint8, float, np.uint8]

def create_environment(game_type: type(Nim) | type(Nimby), solving_algorithm: type(Negamax) | type(DUAL) | type(SSS), in_order: bool, look_up_tables: Optional[TranspositionTable] = None, **kwargs) -> Callable[[DepthConfiguration], DataframeRecord]:
    if look_up_tables is None:
        look_up_tables: TranspositionTable = TranspositionTable()
    
    def play_game(depths: DepthConfiguration) -> DataframeRecord:
        player1_depth, player2_depth = depths
        ai_1: solving_algorithm = solving_algorithm(depth=player1_depth, tt=look_up_tables)
        ai_2: solving_algorithm = solving_algorithm(depth=player2_depth, tt=look_up_tables)
        game_config: dict[str, Any] = {
            'players': [
                AI_Player(ai_1),
                AI_Player(ai_2)
            ],
        } | kwargs
        
        environment: game_type = game_type(**game_config)
        
        if not in_order:
            environment.switch_player()
    
        starting_player: int = environment.current_player
    
        start: float = perf_counter()
        history: list = environment.play()
        end: float = perf_counter()
        
        output: DataframeRecord = str(game_type.__name__), str(solving_algorithm.__name__), np.asarray(depths), np.uint8(starting_player),  np.uint8(environment.current_player), end - start, np.uint8(len(history) - 1)

        print(
            '======== Finished(game_type=\'{}\', solving_algorithm=\'{}\')(depths={}, starting_player={}, winner={}, time={}s, rounds_number={}) ========'.format(
                *output   
            )
        )
        return output
    
    return play_game

def add_to_dataframe(data: pd.DataFrame, records: list[DataframeRecord]) -> None:
    for record in records:
        data.loc[len(data)] = record

results: map = map(create_environment(Nim, Negamax, True, tt), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
results: map = map(create_environment(Nim, Negamax, False, tt), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
df.dtypes

In [ ]:
df

### Negamax - Non-Deterministic

In [ ]:
from itertools import chain

repeat_count: Final[int] = 5

config_nimby: list = list(chain.from_iterable([[element] * repeat_count for element in config_nim]))

config_nimby

In [ ]:
results: map = map(create_environment(Nimby, Negamax, True, tt), config_nimby)
add_to_dataframe(df, list(results))

In [ ]:
results: map = map(create_environment(Nimby, Negamax, False, tt), config_nimby)
add_to_dataframe(df, list(results))

In [ ]:
df

### DUAL - Deterministic

In [ ]:
results: map = map(create_environment(Nim, DUAL, True), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
results: map = map(create_environment(Nim, DUAL, False), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
df

### DUAL - Non-Deterministic

In [ ]:
results: map = map(create_environment(Nimby, DUAL, True), config_nimby)
add_to_dataframe(df, list(results))

In [ ]:
results: map = map(create_environment(Nimby, DUAL, False), config_nimby)
add_to_dataframe(df, list(results))

In [ ]:
df

### SSS - Deterministic

In [ ]:
results: map = map(create_environment(Nim, SSS, True, tt), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
results: map = map(create_environment(Nim, SSS, False, tt), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
df

### SSS - Non-Deterministic

In [ ]:
results: map = map(create_environment(Nimby, SSS, True, tt), config_nimby)
add_to_dataframe(df, list(results))

In [ ]:
results: map = map(create_environment(Nimby, SSS, False, tt), config_nimby)
add_to_dataframe(df, list(results))

In [ ]:
df

## Expecti-minimax

In [ ]:
"""
The standard AI algorithm of easyAI is Negamax with alpha-beta pruning
and (optionnally), transposition tables.
"""

LOWERBOUND, EXACT, UPPERBOUND = -1, 0, 1
inf = float("infinity")


def custom_negamax(game, depth, origDepth, scoring, alpha=+inf, beta=-inf, tt=None):
    """
    This implements Negamax with transposition tables.
    This method is not meant to be used directly. See ``easyAI.Negamax``
    for an example of practical use.
    This function is implemented (almost) according to
    http://en.wikipedia.org/wiki/Negamax
    """

    alphaOrig = alpha

    # Is there a transposition table and is this game in it ?
    lookup = None if (tt is None) else tt.lookup(game)

    if lookup is not None:
        # The game has been visited in the past

        if lookup["depth"] >= depth:
            flag, value = lookup["flag"], lookup["value"]
            if flag == EXACT:
                if depth == origDepth:
                    game.ai_move = lookup["move"]
                return value
            elif flag == LOWERBOUND:
                alpha = max(alpha, value)
            elif flag == UPPERBOUND:
                beta = min(beta, value)

            if alpha >= beta:
                if depth == origDepth:
                    game.ai_move = lookup["move"]
                return value

    if (depth == 0) or game.is_over():
        # NOTE: the "depth" variable represents the depth left to recurse into,
        # so the smaller it is, the deeper we are in the negamax recursion.
        # Here we add 0.001 as a bonus to signify that victories in less turns
        # have more value than victories in many turns (and conversely, defeats
        # after many turns are preferred over defeats in less turns)
        return scoring(game) * (1 + 0.001 * depth)

    if lookup is not None:
        # Put the supposedly best move first in the list
        possible_moves = game.possible_moves()
        possible_moves.remove(lookup["move"])
        possible_moves = [lookup["move"]] + possible_moves

    else:

        possible_moves = game.possible_moves()

    state = game
    best_move = possible_moves[0]
    if depth == origDepth:
        state.ai_move = possible_moves[0]

    bestValue = -inf
    unmake_move = hasattr(state, "unmake_move")

    for move in possible_moves:

        if not unmake_move:
            game = state.copy()  # re-initialize move

        game.make_move(move)
        game.switch_player()

        move_alpha = -custom_negamax(game, depth - 1, origDepth, scoring, -beta, -alpha, tt)

        if unmake_move:
            game.switch_player()
            game.unmake_move(move)

        # bestValue = max( bestValue,  move_alpha )
        if bestValue < move_alpha:
            bestValue = move_alpha
            best_move = move

        if alpha < move_alpha:
            alpha = move_alpha
            # best_move = move
            if depth == origDepth:
                state.ai_move = move
            if alpha >= beta:
                break

    if tt is not None:

        assert best_move in possible_moves
        tt.store(
            game=state,
            depth=depth,
            value=bestValue,
            move=best_move,
            flag=UPPERBOUND
            if (bestValue <= alphaOrig)
            else (LOWERBOUND if (bestValue >= beta) else EXACT),
        )

    return bestValue


class CustomNegamax:
    """
    This implements Negamax on steroids. The following example shows
    how to set up the AI and play a Connect Four game:

        >>> from easyAI.games import ConnectFour
        >>> from easyAI import Negamax, Human_Player, AI_Player
        >>> scoring = lambda game: -100 if game.lose() else 0
        >>> ai_algo = Negamax(8, scoring) # AI will think 8 turns in advance
        >>> game = ConnectFour([Human_Player(), AI_Player(ai_algo)])
        >>> game.play()

    Parameters
    -----------

    depth:
      How many moves in advance should the AI think ?
      (2 moves = 1 complete turn)

    scoring:
      A function f(game)-> score. If no scoring is provided
         and the game object has a ``scoring`` method it will be used.

    win_score:
      Score above which the score means a win. This will be
        used to speed up computations if provided, but the AI will not
        differentiate quick defeats from long-fought ones (see next
        section).

    tt:
      A transposition table (a table storing game states and moves)
      scoring: can be none if the game that the AI will be given has a
      ``scoring`` method.

    Notes
    -----

    The score of a given game is given by

    >>> scoring(current_game) - 0.01*sign*current_depth

    for instance if a loss is -100 points, then losing after 4 moves
    will score -99.96 points but losing after 8 moves will be -99.92
    points. Thus, the AI will choose the move that leads to defeat in
    8 turns, which makes it more difficult for the (human) opponent.
    This will not always work if a ``win_score`` argument is provided.

    """

    def __init__(self, depth: int, scoring=None, win_score=+inf, tt=None):
        self.scoring = scoring
        self.depth: int = depth
        self.tt = tt
        self.win_score = win_score

    def __call__(self, game):
        """
        Returns the AI's best move given the current state of the game.
        """

        scoring = (
            self.scoring if self.scoring else (lambda g: g.scoring())
        )  # horrible hack

        self.alpha = custom_negamax(
            game,
            self.depth,
            self.depth,
            scoring,
            -self.win_score,
            +self.win_score,
            self.tt,
        )
        return game.ai_move


### Expecti-minimax z odcięciem alfa-beta - Deterministic

In [ ]:
results: map = map(create_environment(Nim, CustomNegamax, True, tt), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
results: map = map(create_environment(Nim, CustomNegamax, False, tt), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
df

### Expecti-minimax z odcięciem alfa-beta - Non-Deterministic

In [ ]:
results: map = map(create_environment(Nimby, CustomNegamax, True, tt), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
results: map = map(create_environment(Nimby, CustomNegamax, False, tt), config_nim)
add_to_dataframe(df, list(results))

In [ ]:
df

### Analysis

Added average time spend on computing the move, by all AI players.

In [ ]:
df

In [ ]:
df['avg_round_time'] = (df['time'] / df['rounds']).astype(np.float32)

In [ ]:
print(f'Total computation time: {df["time"].sum()}s.')
print(f'Average computation time: {df["avg_round_time"].mean()}s.')

In [ ]:
df

In [ ]:
player1_nim_wins: Final[int] = len(df.where((df['winner'] == 1) & (df['game_variant'] == 'Nim')).dropna())
player2_nim_wins: Final[int] = len(df.where((df['winner'] == 2) & (df['game_variant'] == 'Nim')).dropna())

In [ ]:
print(f'Player 1 wins: {player1_nim_wins} times in Nim games.')
print(f'Player 2 wins: {player2_nim_wins} times in Nim games.')
print(f'Number of performed Nim games: {player1_nim_wins + player2_nim_wins}.')

In [ ]:
player1_nimby_wins: Final[int] = len(df.where((df['winner'] == 1) & (df['game_variant'] == 'Nimby')).dropna())
player2_nimby_wins: Final[int] = len(df.where((df['winner'] == 2) & (df['game_variant'] == 'Nimby')).dropna())

In [ ]:
print(f'Player 1 wins: {player1_nimby_wins} times in Nimby games.')
print(f'Player 2 wins: {player2_nimby_wins} times in Nimby games.')
print(f'Number of performed Nimby games: {player1_nimby_wins + player2_nimby_wins}.')

In [ ]:
print(f'Total number of games: {len(df)}.')

print(f'Player 1 wins: {player1_nim_wins + player1_nimby_wins} times. Which is {((player1_nim_wins + player1_nimby_wins) / len(df)) * 100:.2f}% of all games.')
print(f'Player 2 wins: {player2_nim_wins + player2_nimby_wins} times. Which is {((player2_nim_wins + player2_nimby_wins) / len(df)) * 100:.2f}% of all games.')

Save the results to a `.csv` file.

In [ ]:
df.to_csv('results.csv', index=False)